# Syrtis Demonstration - Martian Environment
# Heat Loss from an ISS Module on the Martian Surface

This example demonstrates the full thermal solution system of Syrtis, working as intended on the surface of Mars. Our Habitat design is taken directly from the ISS, which uses a thin aluminium wall with multiple layers of thin polymer to act as a Whipple MMOD shield. Details of this shield can be found in this paper [https://ntrs.nasa.gov/api/citations/20060026214/downloads/20060026214.pdf]. We can place the Destiny laboratory directly on the Martian surface to see how it handles the radically different thermal environment.

Many proposals for human Mars missions (noteably, Mars Direct) have the surface habitat transit to Mars without the benefit of a closed aeroshell. As such, the MMOD protection will be retained in our model. However, for a horizontally-oriented module we must

In [1]:
from syrtis import *
from syrtis.configuration import Configuration
from syrtis.solver import Solver
from syrtis.material import Solid, ConstrainedIdealGas
from syrtis.habitat import Habitat
import numpy as np

In [2]:
# Materials definitions
# The absorbivity of perlite and air are assumed as 0.5 but are not required
painted_steel = Solid("Painted steel", 15, 8700, 500, 0.1)
aluminium = Solid("Aluminium", 247, 2700, 900, 0.15)
plastic = Solid("Generic plastic", 10, 1300, 1420, 0.5)

internal_air = ConstrainedIdealGas("STP CO2", 101325, 29, 0.71, 10.9e-6, 749, 0.0153)
martian_air = ConstrainedIdealGas("Martian ambient pressure CO2", 580, 44, 0.71, 10.9e-6, 749, 0.0153)

equator = Configuration("Martian equator at noon",
 "constant temperature", 210, 0.1, 0.29, 210, 580, 1, "cross", 90, 90, 605, T_habitat=290)

In [3]:
destiny_vertical = Habitat(orientation="vertical", length=8.4, endcap_type="flat")

destiny_vertical.create_static_shell(internal_air, 2.00)
destiny_vertical.create_static_shell(aluminium, 4.8e-3)

destiny_vertical.create_static_shell(martian_air, 50e-3)
destiny_vertical.create_static_shell(plastic, 12e-3)
destiny_vertical.create_static_shell(martian_air, 50e-3)

destiny_vertical.create_static_shell(aluminium, 2e-3)

In [7]:
cm = ConfigurationManager(destiny_vertical, equator, {"T_air":[200, 210, 220]})
print(cm.each_configuration_inputs_dicts)
a,b,c = cm.run_all_configurations(verbose=True)
print(c)


[{'name': 'Martian equator at noon', 'solution_type': 'constant temperature', 'T_ground': 210, 'k_ground': 0.1, 'albedo_ground': 0.29, 'T_air': 220, 'p_air': 580, 'v_air': 1, 'air_direction': 'cross', 'solar_altitude': 90, 'solar_azimuth': 90, 'solar_intensity': 605, 'Q_habitat': 0, 'T_habitat': 300, 'GRAVITY': 3.71, 'air': <syrtis.material.ConstrainedIdealGas object at 0x00000197D652B748>}, {'name': 'Martian equator at noon', 'solution_type': 'constant temperature', 'T_ground': 210, 'k_ground': 0.1, 'albedo_ground': 0.29, 'T_air': 220, 'p_air': 580, 'v_air': 1, 'air_direction': 'cross', 'solar_altitude': 90, 'solar_azimuth': 90, 'solar_intensity': 605, 'Q_habitat': 0, 'T_habitat': 300, 'GRAVITY': 3.71, 'air': <syrtis.material.ConstrainedIdealGas object at 0x00000197D652B748>}, {'name': 'Martian equator at noon', 'solution_type': 'constant temperature', 'T_ground': 210, 'k_ground': 0.1, 'albedo_ground': 0.29, 'T_air': 220, 'p_air': 580, 'v_air': 1, 'air_direction': 'cross', 'solar_alti